<a href="https://colab.research.google.com/github/pagarsky/nn2022/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота №5
#### з дисципліни "Нейронні мережі"
#### студента групи КН-31/2
#### Пагарського Олексія
#### Варіант №7

In [11]:
N = 7
i1, i2, i3 = N - 1, N + 29, N + 59

In [23]:
import random
import os
import pathlib
from distutils.dir_util import copy_tree

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### На основі документу вирішити завдання класифікації зображень їжі для 3 класів з набору даних food101

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

!tar xvf food-101.tar.gz

In [ ]:
!ls -la food-101/images

classes = []
for dirpath, dirnames, filenames in os.walk("food-101/images/"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    classes.append(dirpath)

In [28]:
dirs = [
    classes[i1],
    classes[i2],
    classes[i3]
]
classes = [dir.split('/')[-1] for dir in dirs]
print(classes)

['sushi', 'ravioli', 'seaweed_salad']


In [30]:
for cls in classes:
    # print(cls)
    for img in os.walk(f'food-101/images/{cls}/'):
        copy_tree(f'food-101/images/{cls}', f'./images/{cls}/')

In [35]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    './images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_data = train_datagen.flow_from_directory(
    './images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [36]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(20, 3, activation='selu', input_shape=(224, 224, 3)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(15, 5, activation='selu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(10, 7, activation='selu'),
  tf.keras.layers.MaxPool2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3, activation='sigmoid')
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 20)      560       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 20)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 107, 107, 15)      7515      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 15)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 47, 47, 10)        7360      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 10)       0

In [38]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

history = model.fit(
    train_data,
    epochs=10,
    batch_size=64,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=len(val_data),
    callbacks=callbacks,
)

print('\n'*5)
loss, accuracy = model.evaluate(val_data)
print(f'loss: {loss}\naccuracy: {accuracy}')

Epoch 1/10
75/75 [==============================] - 39s 526ms/step - loss: 0.6328 - accuracy: 0.7296 - val_loss: 0.5676 - val_accuracy: 0.7767 - lr: 0.0010
Epoch 2/10
75/75 [==============================] - 39s 524ms/step - loss: 0.6581 - accuracy: 0.7117 - val_loss: 0.6937 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/10
75/75 [==============================] - 39s 524ms/step - loss: 0.6579 - accuracy: 0.7158 - val_loss: 0.6255 - val_accuracy: 0.7650 - lr: 0.0010
Epoch 4/10
75/75 [==============================] - ETA: 0s - loss: 0.6219 - accuracy: 0.7262
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
75/75 [==============================] - 39s 523ms/step - loss: 0.6219 - accuracy: 0.7262 - val_loss: 0.6052 - val_accuracy: 0.7450 - lr: 0.0010
Epoch 5/10
75/75 [==============================] - 39s 525ms/step - loss: 0.5802 - accuracy: 0.7521 - val_loss: 0.5495 - val_accuracy: 0.7733 - lr: 1.0000e-04
Epoch 6/10
75/75 [==============================] - 39s 

### Висновок

При виконанні Ларбораторної роботи 5, ми підготували тренувальний та валідаційний датасети для класифікації їжі, аугментувавши зображення для отримання кращої точності моделей, так як навчальний датасет доволі малий за розміром (800 зображень для кожного класу). Застосували згорткові нейронні мережі для класифікації зображень, отримавши точність ~77% для 3-х вибраних за варіантом класів.